In [317]:
import pandas as pd
import numpy as np
import math
from sys import argv
## read data
X = pd.read_csv("X_train", index_col = 0)#.to_numpy()
#X = X.loc[:, (~(X == 0)).any()]
y = pd.read_csv("Y_train", index_col = 0).to_numpy()
#print(X["own business or self employed"])

print(X.columns.get_loc(' White'))
X = X.to_numpy()
X.shape[1]

177


510

In [318]:
tmp = np.concatenate([X[:,126].reshape(-1,1), X[:,507].reshape(-1,1), X[:,0].reshape(-1,1), X[:,358].reshape(-1,1), X[:,210].reshape(-1,1),X[:,212].reshape(-1,1)],axis = 1)
tmp2 = np.concatenate([X[:,128].reshape(-1,1), X[:,177].reshape(-1,1),X[:,414].reshape(-1,1)\
                     , X[:,495].reshape(-1,1), X[:,457].reshape(-1,1),X[:,180].reshape(-1,1),X[:,113].reshape(-1,1), X[:,418].reshape(-1,1), X[:,120].reshape(-1,1)], axis = 1)


In [319]:

X = np.concatenate((X, tmp**2, tmp**3, tmp**4, tmp**5, tmp[:,0:5] * tmp[:,1:], tmp[:,0:5:2] * tmp[:,1:6:2],tmp2[:,0:8] * tmp2[:,1:]), axis = 1)
X.shape[1]

550

In [320]:


##normalize the data
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)

for row in range(X.shape[0]):
    X[row] = (X[row] - X_mean) / (X_std + 1e-7)
    
##add bias term
X = np.concatenate((np.ones([X.shape[0],1]), X), axis = 1)

In [321]:
##split the data
X_train = X[: math.floor(len(X) * 0.8)] 
X_val = X[math.floor(len(X) * 0.8):]

y_train = y[:math.floor(len(y) * 0.8)]
y_val = y[math.floor(len(y) * 0.8):]

print("shape of training data:", X_train.shape)
print("data to train:", X_train)
#input("Press Enter to continue...")



shape of training data: (43404, 551)
data to train: [[ 1.  0.  0. ...  0.  0.  0.]
 [ 1.  1.  0. ...  0.  0.  0.]
 [ 1.  1. -1. ...  0.  0.  0.]
 ...
 [ 1.  0. -1. ...  0.  0.  0.]
 [ 1.  0. -1. ...  0.  0.  0.]
 [ 1.  2. -1. ...  0.  0.  0.]]


In [322]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def computeCost(x, y, theta):
    eps = 1e-7
    
    return np.sum(-(y * np.log(sigmoid(np.dot(x, theta)) + eps) + (1 - y) * np.log(1- sigmoid(np.dot(x, theta))+ eps))) / x.shape[0]



In [335]:
#train
theta = np.zeros([X_train.shape[1], 1])
J = []
lr = 0.002
num_iter = 3000
eps = 1e-7
beta1 = 0.8
beta2 = 0.9
mt = np.zeros([X_train.shape[1], 1])
vt = np.zeros([X_train.shape[1], 1])
from sys import argv
check = False
filename = "clean.npy"
if check:
	theta = np.load(filename)
    
	predict = np.dot(X_val, theta)
	predict[predict>=.5] = 1
	predict[predict<.5] = 0
	print("cost of validation:", computeCost(X_val, y_val, theta))
	print("accuracy:", (predict == y_val).sum() / X_val.shape[0])

	##predict testing data
	X_test = pd.read_csv("X_test", index_col = 0).to_numpy()
	print("shape X_test", X_test.shape)
	tmp = np.concatenate([X_test[:,126].reshape(-1,1), X_test[:,507].reshape(-1,1), X_test[:,0].reshape(-1,1), X_test[:,358].reshape(-1,1), X_test[:,210].reshape(-1,1),X_test[:,212].reshape(-1,1)],axis = 1)
	tmp2 = np.concatenate([X_test[:,128].reshape(-1,1), X_test[:,177].reshape(-1,1),X_test[:,414].reshape(-1,1)\
                     , X_test[:,495].reshape(-1,1), X_test[:,457].reshape(-1,1), X_test[:,418].reshape(-1,1), X_test[:,120].reshape(-1,1)], axis = 1)

	X_test = np.concatenate((X_test, tmp ** 2, tmp** 3, tmp**4, tmp **5,tmp[:,0:5] * tmp[:,1:], tmp[:,0:5:2] * tmp[:,1:6:2], tmp2[:,0:6] * tmp2[:,1:]), axis = 1)
	print(X_test.shape)
	for row in range(X_test.shape[0]):
		X_test[row] = (X_test[row] - X_mean) / (X_std + 1e-7)
	X_test = np.concatenate((np.ones([X_test.shape[0],1]), X_test), axis = 1)
	predict = np.dot(X_test, theta)
	predict[predict>=.5] = 1
	predict[predict<.5] = 0
	output = pd.DataFrame([x for x in range(X_test.shape[0])], columns = ["id"])
	output['label'] = predict
	output['label'] = output['label'].astype(int)
	output.to_csv('predict.csv', index = False)
	#X_test = X_test.loc[:, (~(X_test == 0)).any()].to_numpy()


else:
	for cnt in range(num_iter):
		X_train = X
		y_train = y
		hx = y_train - sigmoid(np.dot(X_train, theta))
		grad = np.sum(-X_train * hx / X_train.shape[0], axis = 0).reshape(-1,1)
		#momentum
		mt = beta1 * mt + (1 - beta1) * grad
		vt = beta2 * vt + (1 - beta2) * (grad ** 2)
		mhat = mt / (1 - beta1 ** num_iter)
		vhat = vt / (1 - beta2 ** num_iter)
		theta = theta - lr * mhat / (np.sqrt(vhat) + eps)
		J.append(computeCost(X_train, y_train, theta))

		print("Cost:", computeCost(X_train, y_train, theta))
	predict = np.dot(X_train, theta)
	predict[predict>=.5] = 1
	predict[predict<.5] = 0
	print("accuracy:", (predict == y_train).sum() / X_train.shape[0])
	np.save(filename, theta)
	













Cost: 0.6670917591187796
Cost: 0.6358479513019022
Cost: 0.6041088771926344
Cost: 0.5742008656152485
Cost: 0.5472145477309928
Cost: 0.5234968508913874
Cost: 0.5029697764404568
Cost: 0.48532834089693694
Cost: 0.4701703696581504
Cost: 0.4570819817636465
Cost: 0.4456819718559065
Cost: 0.43564241244833457
Cost: 0.4266960497688377
Cost: 0.4186322278426505
Cost: 0.41128977100937775
Cost: 0.40454879646558933
Cost: 0.3983208640133459
Cost: 0.3925413292016521
Cost: 0.387162224464798
Cost: 0.3821463174565842
Cost: 0.3774630747889754
Cost: 0.3730850670244159
Cost: 0.3689864999109332
Cost: 0.3651423820860503
Cost: 0.3615286701580139
Cost: 0.35812262357888963
Cost: 0.35490341398804437
Cost: 0.35185270790616446
Cost: 0.3489550107615404
Cost: 0.34619768861102473
Cost: 0.3435706813855671
Cost: 0.34106585747694396
Cost: 0.3386763524145344
Cost: 0.3363958521888973
Cost: 0.33421811599219825
Cost: 0.3321367393144875
Cost: 0.33014517569600876
Cost: 0.3282369977190805
Cost: 0.32640626090650166
Cost: 0.324647

Cost: 0.26750291405419885
Cost: 0.2674922255040525
Cost: 0.26748062894589536
Cost: 0.2674657436909885
Cost: 0.267450545168013
Cost: 0.2674384822892082
Cost: 0.26742821982311293
Cost: 0.26741746751923046
Cost: 0.2674046347256334
Cost: 0.26739069640194546
Cost: 0.26737791477211104
Cost: 0.26736669782044215
Cost: 0.2673563656757889
Cost: 0.26734670003650834
Cost: 0.2673367750108355
Cost: 0.2673239131498325
Cost: 0.26730955702966946
Cost: 0.2672976399336955
Cost: 0.26728809195368025
Cost: 0.2672788313358945
Cost: 0.2672679447546202
Cost: 0.2672552759738276
Cost: 0.26724278789252454
Cost: 0.2672319522599297
Cost: 0.2672222655957416
Cost: 0.2672132899687039
Cost: 0.2672047953090817
Cost: 0.26719441630617063
Cost: 0.2671810279532665
Cost: 0.26716847973443486
Cost: 0.2671590284163986
Cost: 0.2671507477147396
Cost: 0.2671413592513258
Cost: 0.2671298685263865
Cost: 0.26711770716064764
Cost: 0.2671070185199004
Cost: 0.2670976746863177
Cost: 0.2670890825320418
Cost: 0.26708106776820917
Cost: 0.267

Cost: 0.2649539523558956
Cost: 0.26494922036027563
Cost: 0.26494124374493266
Cost: 0.26493318302094787
Cost: 0.2649281570941199
Cost: 0.2649247346812053
Cost: 0.26492075153974587
Cost: 0.2649146614885237
Cost: 0.2649071729609803
Cost: 0.26490076533816903
Cost: 0.2648959321577001
Cost: 0.2648920126724476
Cost: 0.26488873935978063
Cost: 0.2648850068651979
Cost: 0.2648783178700729
Cost: 0.2648697724534472
Cost: 0.26486382765170485
Cost: 0.26486030148680023
Cost: 0.26485685795911645
Cost: 0.26485167418213723
Cost: 0.2648444785248644
Cost: 0.26483772441946096
Cost: 0.2648326479560816
Cost: 0.26482862878524777
Cost: 0.2648253450619773
Cost: 0.26482214150203925
Cost: 0.26481669248948325
Cost: 0.2648083679017995
Cost: 0.26480151857556594
Cost: 0.26479764302594505
Cost: 0.2647945462400668
Cost: 0.26479021393109003
Cost: 0.26478362331232247
Cost: 0.26477659140813475
Cost: 0.2647711598052215
Cost: 0.2647670351192826
Cost: 0.2647636647120394
Cost: 0.26476074217742057
Cost: 0.2647564401809555
Cost:

Cost: 0.2637204745975523
Cost: 0.2637179472197503
Cost: 0.2637174294543013
Cost: 0.2637165649052778
Cost: 0.26371369959932983
Cost: 0.26370916202756967
Cost: 0.2637054796548155
Cost: 0.2637034738513731
Cost: 0.2637024084347459
Cost: 0.2637019948731184
Cost: 0.2637013061306568
Cost: 0.2636978194207242
Cost: 0.2636921945276213
Cost: 0.26368882166491314
Cost: 0.26368801008310233
Cost: 0.26368756337024035
Cost: 0.26368555629793994
Cost: 0.26368140713423616
Cost: 0.2636773094924809
Cost: 0.2636748673102002
Cost: 0.2636736327839866
Cost: 0.26367311948612954
Cost: 0.26367287453128313
Cost: 0.26367072386857104
Cost: 0.2636653526350823
Cost: 0.26366097080461487
Cost: 0.26365966181929823
Cost: 0.26365941840188795
Cost: 0.2636581296919484
Cost: 0.26365452884901264
Cost: 0.2636502476914554
Cost: 0.26364744973996795
Cost: 0.2636460279083966
Cost: 0.2636454161301443
Cost: 0.26364529846079726
Cost: 0.26364405158820003
Cost: 0.26363945762535235
Cost: 0.26363441880707705
Cost: 0.2636324396363229
Cost: 

Cost: 0.26325722190832535
Cost: 0.2632581661799928
Cost: 0.2632570765389048
Cost: 0.263254394294738
Cost: 0.2632526814864444
Cost: 0.26325257061434704
Cost: 0.2632533782901052
Cost: 0.26325486011416127
Cost: 0.26325596781864236
Cost: 0.26325422768008216
Cost: 0.2632504564506355
Cost: 0.26324904618079903
Cost: 0.2632501821240883
Cost: 0.26325150976515516
Cost: 0.26325118186319546
Cost: 0.26324878107339833
Cost: 0.26324662894380296
Cost: 0.2632461848172398
Cost: 0.26324683833048995
Cost: 0.26324818640775954
Cost: 0.26324967136288707
Cost: 0.26324899472148466
Cost: 0.26324541645969507
Cost: 0.26324314938887494
Cost: 0.2632437850069441
Cost: 0.2632453314162265
Cost: 0.2632457636587906
Cost: 0.2632439431184833
Cost: 0.26324149246708467
Cost: 0.26324051409024674
Cost: 0.26324092883984435
Cost: 0.26324211490454513
Cost: 0.26324379876858955
Cost: 0.26324432251417473
Cost: 0.2632414180590073
Cost: 0.26323824317212013
Cost: 0.2632381350983641
Cost: 0.2632397060265043
Cost: 0.26324072317767894
Co

Cost: 0.2632072545601207
Cost: 0.26320506847536046
Cost: 0.2632039603867278
Cost: 0.2632044262068963
Cost: 0.2632057913983176
Cost: 0.26320776973560595
Cost: 0.26320922080695036
Cost: 0.26320764956364123
Cost: 0.2632043620652537
Cost: 0.2632037201538368
Cost: 0.2632053979387984
Cost: 0.26320709584119617
Cost: 0.2632069959907305
Cost: 0.2632049268570178
Cost: 0.26320337397670224
Cost: 0.26320347214250117
Cost: 0.26320461069160495
Cost: 0.2632064580229946
Cost: 0.2632082957072549
Cost: 0.26320776774308857
Cost: 0.26320447994266827
Cost: 0.26320285119517106
Cost: 0.2632041129459917
Cost: 0.26320601644924907
Cost: 0.26320659622842857
Cost: 0.2632049148260378
Cost: 0.2632029462572661
Cost: 0.26320261116322646
Cost: 0.2632035230537957
Cost: 0.26320516914185693
Cost: 0.2632071690309323
Cost: 0.2632075602927799
Cost: 0.26320471469973244
Cost: 0.2632022766579919
Cost: 0.2632028535148794
Cost: 0.263204789944756
Cost: 0.2632059804800148
Cost: 0.26320496172798685
Cost: 0.2632027976975316
Cost: 0.2

Cost: 0.2631902587461574
Cost: 0.26319173853620054
Cost: 0.2631937827145346
Cost: 0.2631948936226614
Cost: 0.26319282345964023
Cost: 0.263189826329179
Cost: 0.2631896600783881
Cost: 0.26319148333498743
Cost: 0.2631930754606206
Cost: 0.2631927351684624
Cost: 0.26319063508807555
Cost: 0.2631892347422323
Cost: 0.26318949333842767
Cost: 0.26319075102784656
Cost: 0.2631926685047328
Cost: 0.2631944453058842
Cost: 0.26319356326977056
Cost: 0.26319022076109755
Cost: 0.2631889930686607
Cost: 0.2631904484712658
Cost: 0.26319232856193936
Cost: 0.2631926615028597
Cost: 0.2631908155334265
Cost: 0.26318904522880887
Cost: 0.2631889041859344
Cost: 0.2631899363328521
Cost: 0.2631916977606361
Cost: 0.2631936869467789
Cost: 0.26319379338716603
Cost: 0.2631907312393489
Cost: 0.2631885860768796
Cost: 0.2631895058289199
Cost: 0.26319147789836517
Cost: 0.2631924505077917
Cost: 0.26319111758212804
Cost: 0.26318902513832315
Cost: 0.26318840881000294
Cost: 0.2631891786330576
Cost: 0.2631907356937263
Cost: 0.263

Cost: 0.26318689252406
Cost: 0.26318448944092976
Cost: 0.2631816668196356
Cost: 0.2631819017558985
Cost: 0.263183856520787
Cost: 0.2631852480336577
Cost: 0.2631845512053475
Cost: 0.26318237873325917
Cost: 0.2631813033107745
Cost: 0.2631818190986068
Cost: 0.2631832033806151
Cost: 0.26318521548804014
Cost: 0.26318667292112047
Cost: 0.26318515653146307
Cost: 0.26318197464006604
Cost: 0.2631813149558586
Cost: 0.2631829921249097
Cost: 0.26318476659643997
Cost: 0.2631848332070869
Cost: 0.2631828857316479
Cost: 0.26318124026182077
Cost: 0.2631812800166123
Cost: 0.2631824241441674
Cost: 0.2631842596326492
Cost: 0.2631862289542549
Cost: 0.26318595988041976
Cost: 0.2631827251064222
Cost: 0.26318096499172383
Cost: 0.26318213647927996
Cost: 0.2631841205781834
Cost: 0.2631848178440882
Cost: 0.2631832365627095
Cost: 0.2631813005360986
Cost: 0.26318091273269756
Cost: 0.2631818297576007
Cost: 0.26318349803931895
Cost: 0.26318555012052464
Cost: 0.2631861679626449
Cost: 0.26318343403711253
Cost: 0.26318

Cost: 0.263177606099289
Cost: 0.26317960413576597
Cost: 0.26318077696849695
Cost: 0.2631797208926553
Cost: 0.2631776047203369
Cost: 0.2631767987998224
Cost: 0.2631774617115237
Cost: 0.26317896561853565
Cost: 0.2631810418312384
Cost: 0.26318225853683663
Cost: 0.2631803083805445
Cost: 0.26317722408226796
Cost: 0.2631770293617516
Cost: 0.26317888380459276
Cost: 0.2631804949799042
Cost: 0.26318016367968117
Cost: 0.2631780435556166
Cost: 0.2631767165461894
Cost: 0.2631770437755893
Cost: 0.2631783303500485
Cost: 0.26318028692342577
Cost: 0.26318199421979704
Cost: 0.2631810193138489
Cost: 0.26317776887682953
Cost: 0.2631766307053742
Cost: 0.2631781198913301
Cost: 0.2631800106899826
Cost: 0.26318041621835475
Cost: 0.26317865597320333
Cost: 0.2631768262738172
Cost: 0.26317665631897585
Cost: 0.26317768921674484
Cost: 0.2631794403311464
Cost: 0.2631815039497599
Cost: 0.2631817285047652
Cost: 0.2631787095812298
Cost: 0.2631765165332911
Cost: 0.2631773772688175
Cost: 0.26317940035250664
Cost: 0.263

Cost: 0.2631746399474705
Cost: 0.2631740750407437
Cost: 0.2631749147398048
Cost: 0.2631765136725411
Cost: 0.2631786291177909
Cost: 0.2631794882921819
Cost: 0.2631770429272907
Cost: 0.263174271798832
Cost: 0.2631745358683263
Cost: 0.26317649758302797
Cost: 0.2631779011148638
Cost: 0.2631771543442747
Cost: 0.26317502595235753
Cost: 0.2631739776054388
Cost: 0.2631744992159038
Cost: 0.2631759159355451
Cost: 0.2631779436064718
Cost: 0.2631794347901206
Cost: 0.2631779339885185
Cost: 0.26317470369991725
Cost: 0.2631740906456915
Cost: 0.2631758028047038
Cost: 0.263177574559774
Cost: 0.26317758271212627
Cost: 0.2631755741458843
Cost: 0.2631740238948522
Cost: 0.26317414822673724
Cost: 0.2631753333521048
Cost: 0.2631772199194509
Cost: 0.2631791055658866
Cost: 0.26317865595789647
Cost: 0.2631754407037619
Cost: 0.2631738411783308
Cost: 0.2631751042840603
Cost: 0.2631770668831177
Cost: 0.2631777604104459
Cost: 0.26317620564604066
Cost: 0.26317424264443584
Cost: 0.2631738748563384
Cost: 0.26317479774

In [38]:
tmp = pd.read_csv("X_train", index_col = 0)